In [1]:
import pandas as pd
import numpy as np
import os
from aocd import get_data

import sys
import time
from collections import defaultdict, deque
from math import lcm

In [2]:
puzzle = get_data(day=20, year=2023)

file = "/Users/liesbeth_bogaert/Documents/AoC/2023/input/input_day20.txt"
lines = open(file, "w").write(puzzle)

puzzle = puzzle.splitlines()
print(puzzle)

['%vn -> ts, lq', '&ks -> dt', '%zt -> vl', '%xg -> ts, pb', '&xd -> qz, bc, zt, vk, hq, qx, gc', '&pm -> dt', '%gb -> vj, xd', '%qx -> gb', '%rl -> qn', '%lq -> gk', '%qm -> bf', '%zn -> vh, pf', '%lz -> kk, vr', '%bf -> rr', '%gx -> vr', '%zr -> vx, pf', '%lt -> ng, vr', '%hd -> mg, xd', '%mg -> xd', '%tx -> jg, vr', '%gk -> kx, ts', '&vr -> tr, vf, tx, ks, kk, jg', 'broadcaster -> qz, tx, jr, hk', '%bc -> qx', '%xz -> lt, vr', '%jg -> sb', '%qn -> zr, pf', '%gc -> xv', '%vx -> lj, pf', '%vf -> cn', '&dt -> rx', '%sb -> lz, vr', '%kx -> xg', '%hk -> pf, tv', '%cb -> pf', '&dl -> dt', '%vl -> xd, bc', '%fl -> pp, pf', '%ng -> vr, gx', '%jr -> ts, qm', '%cd -> vn, ts', '%mt -> ts', '%rr -> ts, cd', '%tr -> xz', '%hq -> zt', '%xv -> hq, xd', '%vj -> xd, hd', '%pp -> zn', '%vh -> pf, cb', '%cn -> vr, tr', '%kk -> vf', '&pf -> pp, tv, rl, pm, hk', '&ts -> dl, qm, kx, lq, bf, jr', '%tv -> rl', '&vk -> dt', '%pb -> ts, mt', '%lj -> pf, fl', '%qz -> xd, gc']


In [3]:
# Testing with M input file
#file = "/Users/liesbeth_bogaert/Documents/AoC/2023/input/input_day20_m.txt"
#puzzle = open(file).read().splitlines()

In [9]:
conjs = {}
flips = {}
broad = []

rx = ""

for line in puzzle:
    module, dest = line.split(' -> ')
    dest = dest.split(', ')
    type = module[0]
    label = module[1:]

    if "rx" in dest:
        rx = label
    
    if type == '%':
        flips[label] = dest
    elif type == '&':
        conjs[label] = dest
    elif module == 'broadcaster':
        broad = dest
    
#print(conjs)
#print(flips)
#print(broad)

In [40]:
def initialise(): 
    global lows, highs, presses, flip_states, conj_mem

    lows = 0
    highs = 0
    presses = 0

    flip_states = {}
    conj_mem = defaultdict(dict)

    for f in flips.keys():
        flip_states[f] = False
        for o in flips[f]:
            if o in conjs.keys():
                conj_mem[o][f] = False

    for k, v in conjs.items():
        for o in v:
            if o in conjs.keys():
                conj_mem[o][k] = False


In [48]:
def press(part):
    global lows, highs, presses
    presses += 1
    
    pulse_queue = [(False, "button", "broadcaster")]

    while pulse_queue:        
        pulse, src, dst = pulse_queue.pop(0)
        
        if part & (dst == 'rx'): # For part 2
            continue
        
        if pulse == True:
            highs += 1
        else:
            lows += 1
            
        if dst == "broadcaster":
            for b in broad:
                pulse_queue.append((pulse, dst, b))
                
        elif dst in flips.keys():
            if pulse == False:
                flip_states[dst] = not flip_states[dst]
                for f2 in flips[dst]:
                    pulse_queue.append((flip_states[dst], dst, f2))
                    
        elif dst in conjs.keys():
            conj_mem[dst][src] = pulse
            snd = not all(v for v in conj_mem[dst].values())
            for f2 in conjs[dst]:
                pulse_queue.append((snd, dst, f2))
 
        for label, val in conj_mem[rx].items(): # For part 2
                if val == True and label not in rx_cons:
                    rx_cons[label] = presses

In [49]:
# Part 1
initialise()

for _ in range(1000):
    press(False)

#print(presses, lows, highs)
print(lows * highs)

1000 16687 42647
711650489


In [51]:
# Part 2
rx_cons = {}
initialise()

while len(rx_cons) < 4:
    press(True)

print(rx_cons)
print(lcm(*rx_cons.values()))

{'dl': 3769, 'pm': 3833, 'vk': 3877, 'ks': 3917}
219388737656593
